## 1. Analysis of costs to run animal shelters
<p>You are working as a data analyst in the animal welfare department. In preparation for next year's budget, the head of your department would like to know the total cost to shelter animals, broken down by each animal type and size combination (i.e., 'small dogs', 'large dogs', etc.). You need to prepare a table that contains this information.</p>
<p>The total cost to shelter an animal for the year is calculated as the sum of three factors: the size and animal type, the age, and the location.</p>
<ul>
<li>The base cost of sheltering an animal is based upon its size and type. The costs per animal type and size are contained in the <code>size_costs</code> table. The criteria for classifying size has recently been updated, and so you will need to categorize animals based upon the following table:</li>
</ul>
<table>
<thead>
<tr>
<th style="text-align:right;"></th>
<th style="text-align:right;">Small</th>
<th>Medium</th>
<th>Large</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:right;">Dog</td>
<td style="text-align:right;">&lt;= 10 lbs</td>
<td>10 lbs &lt; and &lt;= 30 lbs</td>
<td>30 lbs &lt;</td>
</tr>
<tr>
<td style="text-align:right;">Cat</td>
<td style="text-align:right;">&lt;= 5 lbs</td>
<td>5 lbs &lt; and &lt;= 7 lbs</td>
<td>7 lbs &lt;</td>
</tr>
<tr>
<td style="text-align:right;">Bird</td>
<td style="text-align:right;">&lt;= 0.7 lbs</td>
<td>0.7 lbs &lt; and &lt;= 1.1 lbs</td>
<td>1.1 lbs &lt;</td>
</tr>
</tbody>
</table>
<ul>
<li>Older animals cost more, and so an age cost (contained in the <code>age_costs</code> table) is added. Each animal's age should be calculated as the age by the end of the year (December 31st, 2021).</li>
<li>There is a location cost depending on where the animal is sheltered (contained in the <code>location_costs</code> table). </li>
<li>The calculation should not include animals that have been sponsored by private charities (sponsored animals are listed in the <code>sponsored_animals</code> table).</li>
</ul>
<p>For future visualization purposes, you will also need to include a <code>percentage</code> column in your result. This percentage should reflect the fraction of the total cost to be allocated to each animal and size combination.</p>
<p>The data you need is available in the tables shown in the database schema below.</p>
<h5 id="databaseschema">Database Schema</h5>
<p><img src="https://assets.datacamp.com/production/repositories/5934/datasets/a946a159c0024ee0995f7a030f2c0cf802203835/diagram.PNG" width="500" height="500"> </p>

## Creating a temp table for animals where data manipulation will be done

In [80]:
%%sql
postgresql:///animal_shelters
    create table temp_animals as
    (
    select animalid, animaltype, birthdate, weight, location
    from animals
    )

(psycopg2.errors.DuplicateTable) relation "temp_animals" already exists

[SQL: create table temp_animals as
    (
    select animalid, animaltype, birthdate, weight, location
    from animals
    )]
(Background on this error at: http://sqlalche.me/e/14/f405)


## Adding a new colum 'age'

In [81]:
%%sql
postgresql:///animal_shelters

alter table temp_animals
add column age int

(psycopg2.errors.DuplicateColumn) column "age" of relation "temp_animals" already exists

[SQL: alter table temp_animals
add column age int]
(Background on this error at: http://sqlalche.me/e/14/f405)


# Setting value for the age column
## Age value is the difference between the birth year and the current year (2021)

In [82]:
%%sql
postgresql:///animal_shelters

update temp_animals
set age = 2021 - extract(year from (cast(birthdate as date)))

9757 rows affected.


[]

# Adding a sizeid column to be able to refrence the size_costs cost table 

In [83]:
%%sql
postgresql:///animal_shelters
    alter table temp_animals
    add column sizeid text

(psycopg2.errors.DuplicateColumn) column "sizeid" of relation "temp_animals" already exists

[SQL: alter table temp_animals
    add column sizeid text]
(Background on this error at: http://sqlalche.me/e/14/f405)


## Populating the case id with the specification given

In [84]:
%%sql
postgresql:///animal_shelters
   update temp_animals
    set sizeid = CASE
                WHEN weight <= 10 THEN 'DS'
                WHEN weight > 30 THEN 'DL'
                ELSE 'DM'
                END
    where animaltype = 'Dog'

5908 rows affected.


[]

In [85]:
%%sql
postgresql:///animal_shelters
   update temp_animals
    set sizeid = CASE
                WHEN weight <= 5 THEN 'CS'
                WHEN weight > 7 THEN 'CL'
                ELSE 'CM'
                END
    where animaltype = 'Cat'

3803 rows affected.


[]

In [86]:
%%sql
postgresql:///animal_shelters
   update temp_animals
    set sizeid = CASE
                WHEN weight <= 0.7 THEN 'BS'
                WHEN weight > 1.1 THEN 'BL'
                ELSE 'BM'
                END
    where animaltype = 'Bird'

46 rows affected.


[]

## Checking if changes took place

In [ ]:
%%sql
postgresql:///animal_shelters
   select * from temp_animals

## Since the sponsored animals are not needed in the records, it was removed

In [88]:
%%sql
postgresql:///animal_shelters

DELETE FROM temp_animals
WHERE temp_animals.animalid IN (SELECT sponsored_animals.sponsorid FROM sponsored_animals)


0 rows affected.


[]

## Adding column named age

In [89]:
%%sql
postgresql:///animal_shelters

alter table temp_animals
alter column age type int

Done.


[]

## Changing the age datatype in the age_cost column to enable calculation/ refrencing 

In [90]:
%%sql
postgresql:///animal_shelters

alter table age_costs
alter column age type int
USING age::int

Done.


[]

# Joining the temp_animals table to age_costs, size_costs and location_costs table
## Putting the result into a new table called temp2_animals so that s percentage calculation can be done

In [92]:
%%sql
postgresql:///animal_shelters
 
SELECT temp_animals.animaltype, size, 
sum(size_costs.costs + location_costs.costs + age_costs.costs) as total 
FROM temp_animals
LEFT JOIN size_costs ON temp_animals.sizeid = size_costs.sizeid
LEFT JOIN location_costs ON temp_animals.location = location_costs.location
LEFT JOIN age_costs ON temp_animals.age = age_costs.age
group by temp_animals.animaltype, size

In [ ]:
%%sql
postgresql:///animal_shelters
     
    create table temp2_animals as (
    select temp_animals.animaltype, size,
    sum(size_costs.costs + location_costs.costs + age_costs.costs) as total
    FROM temp_animals
    LEFT JOIN size_costs ON temp_animals.sizeid = size_costs.sizeid
    LEFT JOIN location_costs ON temp_animals.location = location_costs.location
    LEFT JOIN age_costs ON temp_animals.age = age_costs.age
    group by temp_animals.animaltype, size)


## Adding a new column called percentage_deno to be used to store the subtotal of each animal type

In [ ]:
%%sql
postgresql:///animal_shelters
    
alter table temp2_animals
add column percentage_deno DOUBLE PRECISION

In [ ]:
%%sql
postgresql:///animal_shelters

update temp2_animals
set percentage_deno = (select sum(total)
                       from temp2_animals
                       where animaltype = 'Bird')
where animaltype = 'Bird'

In [ ]:
%%sql
postgresql:///animal_shelters

update temp2_animals
set percentage_deno = (select sum(total)
                       from temp2_animals
                       where animaltype = 'Cat')
where animaltype = 'Cat'

In [ ]:
%%sql
postgresql:///animal_shelters

update temp2_animals
set percentage_deno = (select sum(total)
                       from temp2_animals
                       where animaltype = 'Dog')
where animaltype = 'Dog'

## Selecting Columns as specified in the challenge

In [ ]:
%%sql
postgresql:///animal_shelters
select animaltype, size, total,
    CAST((total * 100 / percentage_deno) AS DECIMAL(12,2)) as percentage
from temp2_animals
order by animaltype, size desc